In [86]:
import json
import pandas as pd

alias01 = "edp"
alias02 = "galp"

json01 = {f"{alias01}|{k}": v["date"] for k,v in json.load(open(f"data/kwrd_{alias01}.json")).items() if v["filter"] > 0.02}
json02 = {f"{alias02}|{k}": v["date"] for k,v in json.load(open(f"data/kwrd_{alias02}.json")).items() if v["filter"] > 0.02}

json01.update(json02)

relations = pd.DataFrame(json01)
relations.fillna(0, inplace=True)

relations = relations.corr()
if alias01 != alias02:
    relations = relations.iloc[-len(json02):, :-len(json02)]
relations

,edp|Abr,edp|Acompanhe,edp|AdC,edp|Alemanha,edp|Alerta,edp|Alexandra Lucas Coelho,edp|Alexandre Brito,edp|Alvito,edp|Ana Brito,edp|Angola,...,edp|votos,edp|website,edp|zona,edp|zonas,edp|Água de Mar,edp|Últimas,edp|água,edp|áreas,edp|âmbito,edp|órgão
galp|A Galp Energia,-0.043145,-0.056142,0.014164,0.023792,-0.009235,-0.011845,0.013533,-0.043479,0.280766,0.095875,...,-0.026095,-0.040247,-0.041317,-0.066742,0.117193,0.001030,0.074489,-0.068443,-0.041651,0.020842
galp|AdC,-0.026833,0.057800,-0.006802,-0.025426,-0.036990,-0.023036,-0.020121,-0.047128,0.150354,0.087060,...,-0.045235,-0.023180,-0.066832,-0.039553,0.031311,-0.036742,0.109196,-0.030407,0.014300,-0.041319
galp|Adolfo Mesquita Nunes,-0.027926,-0.008399,-0.021321,0.026873,-0.040207,-0.026072,-0.027488,-0.030741,0.123929,-0.029677,...,0.024400,-0.013655,-0.015577,0.104734,-0.019024,0.238779,0.157145,-0.014200,0.268055,0.144079
galp|Alemanha,-0.045186,0.011894,0.021017,0.162394,0.020371,-0.026354,-0.053176,-0.059469,0.171815,0.240181,...,0.074071,-0.026678,-0.009330,0.027333,0.091628,0.003104,0.053530,-0.004562,-0.018363,0.095324
galp|Alentejo,-0.042597,0.132120,0.054735,0.126464,-0.004470,-0.065577,-0.043743,0.211210,0.190660,-0.010939,...,-0.010623,-0.015691,-0.023099,0.136778,0.141869,0.395299,0.195751,0.061873,0.218085,0.038561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
galp|água,-0.002428,-0.066760,-0.024056,-0.004658,0.101784,-0.024998,0.029856,-0.032724,-0.010691,0.028947,...,-0.057902,-0.024478,0.096535,0.051122,0.075392,0.125911,0.074127,-0.060608,-0.015365,-0.001074
galp|águas,0.035792,-0.086190,0.164214,0.014602,0.094885,0.006824,0.045605,-0.050831,0.014181,0.009548,...,0.006434,-0.030016,0.046503,0.004743,0.287629,0.047704,0.001185,0.020184,0.270092,0.157621
galp|áreas,0.004259,0.171218,0.062751,0.070193,0.049195,-0.041042,0.015662,0.012705,0.066303,0.010715,...,-0.012903,0.129476,0.023006,0.021005,0.014477,0.186215,0.111815,0.135638,0.131652,0.011950
galp|áudio,-0.021670,0.029566,-0.010961,0.018541,-0.013890,-0.038718,0.000110,-0.016588,0.388331,0.022270,...,-0.044951,-0.021013,-0.037308,-0.039130,0.093155,-0.060987,0.027648,-0.047212,0.002057,0.013407


In [87]:
corr_long = relations.stack().reset_index()
corr_long

,level_0,level_1,0
0,galp|A Galp Energia,edp|Abr,-0.043145
1,galp|A Galp Energia,edp|Acompanhe,-0.056142
2,galp|A Galp Energia,edp|AdC,0.014164
3,galp|A Galp Energia,edp|Alemanha,0.023792
4,galp|A Galp Energia,edp|Alerta,-0.009235
...,...,...,...
705907,galp|âmbito,edp|Últimas,0.146504
705908,galp|âmbito,edp|água,0.101604
705909,galp|âmbito,edp|áreas,0.118577
705910,galp|âmbito,edp|âmbito,0.188633


In [88]:
corr_long = corr_long[
    (corr_long[0].abs() >= .825) &
    (corr_long[0].abs() <= 0.975)
]
corr_long['sorted_pair'] = corr_long.apply(lambda row: tuple(sorted([row['level_0'], row['level_1']])), axis=1)


from collections import Counter

# Count the frequency of each word
word_counts = Counter(corr_long['level_0'].tolist() + corr_long['level_1'].tolist())

# Function to compute rarity score for a pair
def rarity_score(row):
    return word_counts[row['level_0']] + word_counts[row['level_1']]

# Calculate rarity score for each pair
corr_long['Rarity'] = corr_long.apply(rarity_score, axis=1)
corr_long = corr_long.sort_values(by='Rarity', ascending=True)

###

corr_long[corr_long["Rarity"] > 1]

/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_13419/2410681354.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corr_long['sorted_pair'] = corr_long.apply(lambda row: tuple(sorted([row['level_0'], row['level_1']])), axis=1)
/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_13419/2410681354.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corr_long['Rarity'] = corr_long.apply(rarity_score, axis=1)


,level_0,level_1,0,sorted_pair,Rarity
659225,galp|shares,edp|shares,0.845000,"(edp|shares, galp|shares)",2
90999,galp|Galpgate,edp|Manuel Pinho,0.903025,"(edp|Manuel Pinho, galp|Galpgate)",2
621619,galp|reforço,edp|Isabel Salema,0.837430,"(edp|Isabel Salema, galp|reforço)",2
33175,galp|Bruxelas,edp|Repsol,0.843034,"(edp|Repsol, galp|Bruxelas)",3
100928,galp|Gémeas Cargas,edp|emissões,0.859388,"(edp|emissões, galp|Gémeas Cargas)",3
206839,galp|Tribunal da Concorrência,edp|Repsol,0.893355,"(edp|Repsol, galp|Tribunal da Concorrência)",3
100619,galp|Gémeas Cargas,edp|Gémeas Cargas,0.894048,"(edp|Gémeas Cargas, galp|Gémeas Cargas)",4
116713,galp|Jul,edp|Leia,0.826911,"(edp|Leia, galp|Jul)",4
490532,galp|localização,edp|Entrar,0.840262,"(edp|Entrar, galp|localização)",5
35567,galp|CESE,edp|Polícia Judiciária,0.923701,"(edp|Polícia Judiciária, galp|CESE)",5
